
# Задание:
----

<br>

Вам предоставлен анонимизированные результаты **реального A/B тестирования**, проходившего когда-то в нашем отделе (`ab_test_ensembles.csv`). В тесте было два варианта работы сервиса - `standard` и `svm_ensemble_v_1_22`, где работали несколько моделей классификации для целей сервиса.

<br>


------
Вам, как специалистам по машинному обучению предстоит ответить на **2 главных вопроса:**

1. Стоит ли нам оставить старый вариант работы сервиса или заменить его на вариант работы с моделями классификации (используем всю выборку 200к+ пользователей).
2. Кроме того, посчитайте вывод для типа пользователей (`user_type`). Стоит ли для новых (старых) пользователей оставить старый (новый) вариант работы сервиса.


<br>

Для того, чтобы освежить в памяти процесс тестирования статистических гипотез, непомню, что тестирование состоит из следующих частей:
1. Дизайн эксперимента.
2. Подготовка и запуск эксперимента.
3. Сбор данных и аналитика полученных данных.
4. Визуализация результатов тестирования.
5. Тестирование гипотез.
6. Вывод и интерпритация результатов.

-----

#### Критерии оценки задания:

1. **Первое, что будет проверяться - вывод полученных результатов**, в случае если выводы сделаны не правильно, задание считается проваленным и на этом этап проверки заканчивается (пропускаются этапы код-ревью, оформления и визуализации, качество кода).
2. В случае если результаты и интерпритация результатов оказались верны, проводится код-ревью и проверка этапа визуализаций, поиск проблемных точек, точек роста.
3. **Максимальный балл** который можно получить, выполнив текущее задание: **2 балла за 1 вопрос, 2 балла за 2 вопрос и 1 балл за эффективный и аккуратный код.


Удачи, примените все свои навыки, которые вам доступны на данный момент и покажите на что вы способны!

In [73]:
import pandas as pd
import numpy as np
#import statistics
#from scipy.stats import stats, norm, binom, beta, normaltest, probplot

#import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

In [74]:
df = pd.read_csv('./ab_test_ensembles.csv')
df

,user_id,timestamp,group,variants,converted,location,age_group,user_type
0,9109b0dc-d393-497f-8d63-ba9a25dd16b4,2022-05-21 22:11:48.556739,control,standard,0,United Kingdom,18-25,registered_user
1,2430c3d2-f75b-4b31-8271-51b6a76c2652,2022-05-12 08:01:45.159739,control,standard,0,United Kingdom,42-49,registered_user
2,44788c4e-8dd2-4fad-b986-75e76f4adb64,2022-05-11 16:55:06.154213,treatment,svm_ensemble_v_1_22,0,United Kingdom,26-33,new_user
3,4699a417-506d-41b8-a354-6af6ad576963,2022-05-08 18:28:03.143765,treatment,svm_ensemble_v_1_22,0,United Kingdom,42-49,registered_user
4,304b0d28-bcdf-401a-9dff-66230d3ba0bc,2022-05-21 01:52:26.210827,control,standard,1,United Kingdom,42-49,registered_user
...,...,...,...,...,...,...,...,...
294473,497bf0f8-0092-4736-8ae7-775b6cc9736c,2022-05-03 22:28:38.630509,control,standard,0,United Kingdom,58+,new_user
294474,6cb20e62-e49d-41cf-97da-0e11cb439dca,2022-05-12 00:51:57.078372,control,standard,0,United Kingdom,42-49,new_user
294475,d307b0ad-92a1-409c-a2d2-da8f4a118576,2022-05-22 11:45:03.439544,control,standard,0,United Kingdom,18-25,registered_user
294476,c6bd8da5-2114-4fd4-92c6-e4b11a8be4dc,2022-05-15 01:20:28.957438,control,standard,0,United Kingdom,50-57,new_user


In [75]:
#import numpy as np
#import pandas as pd
#import scipy.stats as stats
#import statsmodels.stats.api as sms
#import seaborn as sns
#from math import ceil

In [76]:
#    Заменим значения в group на A и B, в user_type - 0 и 1
df['group'] = np.where(df['group'] == 'control', 'A', 'B')
df['user_type'] = np.where(df['user_type'] == 'registered_user', 0, 1)

In [77]:
#    Посчитаем неуникальных пользователей
nuniq_users = df['user_id'].value_counts(ascending = False)
nuniq_users[nuniq_users > 1].count()

0

In [78]:
#   Если бы неуникальные были, то удалим вот так:
users_to_drop = nuniq_users[nuniq_users > 1].index
df = df[~ df['user_id'].isin(users_to_drop)]

## Проверка гипотез

### Признак 'converted' по всем типам пользователей

Нулевая гипотеза: нет статистически значимой разницы между пользователями A и B по признаку 'converted' 
Альтернативная гипотеза: есть статистически значимая разница между пользователями A и B по признаку 'converted'

Делать выводы будем исходя из z-test

In [79]:
#    функция для формирования сэмплов
def sampling(n_sam, data, feature_AB):  
    A_sample = data[data[feature_AB] == 'A'].sample(n = n_sam, random_state = 42)
    B_sample = data[data[feature_AB] == 'B'].sample(n = n_sam, random_state = 42)
    AB_sample = pd.concat([A_sample, B_sample], axis=0)
    AB_sample.reset_index(drop=True, inplace=True)
    
    return AB_sample

In [80]:
#    подсчет p-value
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
def pval_calc(feature_AB, feature_0_1, AB_sample):

    A_results = AB_sample[AB_sample[feature_AB] == 'A'][feature_0_1]
    B_results = AB_sample[AB_sample[feature_AB] == 'B'][feature_0_1]

    num_A = A_results.count()
    num_B = B_results.count()
    A_B = [A_results.sum(), B_results.sum()]
    nobs = [num_A, num_B]

    z_stat, pval = proportions_ztest(A_B, nobs = nobs)
    (lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(A_B, nobs=nobs, alpha=0.05)

    print(f'Feature: {feature_0_1}')
    print(f'z-stat: {z_stat:.2f}')
    print(f'p-value: {pval:.3f}')
    print(f'CI 95% for A group - [{lower_con:.4f}, {upper_con:.4f}]')
    print(f'CI 95% for B group - [{lower_treat:.4f}, {upper_treat:.4f}]\n')

In [81]:
pval_calc('group', 'converted', sampling(20000, df, 'group'))

Feature: converted
z-stat: 1.34
p-value: 0.181
CI 95% for A group - [0.1213, 0.1304]
CI 95% for B group - [0.1169, 0.1260]



p-value > 0.05, следовательно отклонять нулевую гипотезу нельзя.
Иными словами, показатель конверсии (converted) не имеет статистически значимой разницы при вводимых изменениях для группы пользователей control и treatment. Вводить в продакшн новую фичу не стоит.

### Признак 'converted' для каждого 'user_type'

Тут мы разделим наш датасет по признаку new/registered user и будем вычислять p-value для отдельный категорий пользователей.

In [82]:
df['user_type'].value_counts(normalize=True)

user_type
0    0.500003
1    0.499997
Name: proportion, dtype: float64

In [83]:
df_new_user = df[df['user_type'] == 1]
df_old_user = df[df['user_type'] == 0]

AB_sample_new_user = sampling(20000, df_new_user, 'group')
AB_sample_old_user = sampling(20000, df_old_user, 'group')

pval_calc('group', 'converted', AB_sample_new_user)
pval_calc('group', 'converted', AB_sample_old_user)

Feature: converted
z-stat: -0.15
p-value: 0.877
CI 95% for A group - [0.1134, 0.1223]
CI 95% for B group - [0.1139, 0.1228]

Feature: converted
z-stat: -1.08
p-value: 0.280
CI 95% for A group - [0.1127, 0.1217]
CI 95% for B group - [0.1162, 0.1252]



p-value - и для новых пользователей, и для старых - > 0.05. Также нет статистически значимых различий ни для одной из групп в type_user. Не стоит оставлять новую фичу ни для старых пользователей, ни для новых.